# mom_app Project Creator Notebook (v2)

Run cells to create the project and zip it.

## Section 1 — Create folders

In [ ]:
import os
base='/mnt/data/mom_app'
for d in ['agents','utils','output','uploads']:
    os.makedirs(os.path.join(base,d), exist_ok=True)
print('Created', base)

## Section 2 — Write project files

In [ ]:
import os, json
base='/mnt/data/mom_app'
files = {"utils/llm_config.py": "import os, json\nclass LLMAdapter:\n    def __init__(self, backend=None, model_overrides=None):\n        self.backend = backend or os.getenv('LLM_BACKEND','dummy')\n    def generate(self,prompt:str)->str:\n        return 'DUMMY:'+prompt[:200]\n", "utils/file_handler.py": "def ingest_any(path):\n    with open(path,'r',encoding='utf-8',errors='ignore') as f:\n        return f.read()\n", "utils/export_docs.py": "import os\nfrom docx import Document\nfrom docx.shared import Pt\nfrom reportlab.lib.pagesizes import A4\nfrom reportlab.pdfgen import canvas\n\ndef export_to_docx_pdf(text,output_dir):\n    os.makedirs(output_dir,exist_ok=True)\n    docx_path=os.path.join(output_dir,'Minutes_of_Meeting_FINAL.docx')\n    pdf_path=os.path.join(output_dir,'Minutes_of_Meeting_FINAL.pdf')\n    doc=Document()\n    style=doc.styles['Normal']\n    style.font.name='Calibri'\n    style.font.size=Pt(11)\n    for line in text.splitlines():\n        doc.add_paragraph(line)\n    doc.save(docx_path)\n    c=canvas.Canvas(pdf_path,pagesize=A4)\n    width,height=A4\n    y=height-50\n    for line in text.splitlines():\n        if y<50:\n            c.showPage(); y=height-50\n        c.drawString(50,y,line[:110]); y-=14\n    c.save()\n    return docx_path,pdf_path\n", "agents/extract_agent.py": "from utils.file_handler import ingest_any\nfrom utils.llm_config import LLMAdapter\nimport os,json\n\ndef extract_content(path):\n    text=ingest_any(path)\n    llm=LLMAdapter(os.getenv('LLM_BACKEND','dummy'))\n    raw=llm.generate('extract:'+text[:500])\n    try:\n        return json.loads(raw)\n    except Exception:\n        return {'raw_text':text,'llm':raw}\n", "agents/summarize_agent.py": "from utils.llm_config import LLMAdapter\nimport os,json\n\ndef summarize_content(extracted):\n    llm=LLMAdapter(os.getenv('LLM_BACKEND','dummy'))\n    return llm.generate('summarize:'+json.dumps(extracted)[:500])\n", "agents/critique_agent.py": "from utils.llm_config import LLMAdapter\nimport os,json\n\ndef critique_summary(draft):\n    llm=LLMAdapter(os.getenv('LLM_BACKEND','dummy'))\n    raw=llm.generate('critique:'+draft[:500])\n    try:\n        return json.loads(raw)\n    except Exception:\n        return {'raw':raw}\n", "agents/aggregate_agent.py": "from utils.llm_config import LLMAdapter\nimport os,json\n\ndef aggregate_content(draft,critique):\n    llm=LLMAdapter(os.getenv('LLM_BACKEND','dummy'))\n    return llm.generate('aggregate:'+draft[:500]+'|'+json.dumps(critique)[:200])\n", "agents/polish_agent.py": "from utils.llm_config import LLMAdapter\nimport os\n\ndef polish_final(text):\n    backend=os.getenv('LLM_FINAL_BACKEND','same')\n    if backend=='same': backend=os.getenv('LLM_BACKEND','dummy')\n    llm=LLMAdapter(backend)\n    return llm.generate('polish:'+text[:500])\n", "pipeline.py": "from agents.extract_agent import extract_content\nfrom agents.summarize_agent import summarize_content\nfrom agents.critique_agent import critique_summary\nfrom agents.aggregate_agent import aggregate_content\nfrom agents.polish_agent import polish_final\nfrom utils.export_docs import export_to_docx_pdf\nimport os,uuid\n\ndef run_pipeline(file_path,session_id=None):\n    session_id=session_id or str(uuid.uuid4())[:8]\n    outdir=os.path.join(os.getenv('OUTPUT_DIR','/mnt/data/output'),session_id)\n    os.makedirs(outdir,exist_ok=True)\n    extracted=extract_content(file_path)\n    draft=summarize_content(extracted)\n    critique=critique_summary(draft)\n    aggregated=aggregate_content(draft,critique)\n    polished=polish_final(aggregated)\n    docx,pdf=export_to_docx_pdf(polished,outdir)\n    return docx,pdf,outdir\n", "app.py": "import streamlit as st\nimport uuid,os,time\nfrom pipeline import run_pipeline\nst.set_page_config(page_title='MoM Gen')\nst.title('MoM Generator')\nuploaded=st.file_uploader('Upload',type=['txt'])\nif uploaded:\n    sid=str(uuid.uuid4())[:8]\n    up='/mnt/data/mom_app/uploads'\n    os.makedirs(up,exist_ok=True)\n    path=os.path.join(up,f\"{sid}_{uploaded.name}\")\n    with open(path,'wb') as f: f.write(uploaded.getbuffer())\n    st.text('Processing...')\n    docx,pdf,out=run_pipeline(path,sid)\n    st.success('Done')\n    with open(docx,'rb') as f: st.download_button('DOCX',f,file_name='MoM.docx')\n    with open(pdf,'rb') as f: st.download_button('PDF',f,file_name='MoM.pdf')\n", "requirements.txt": "streamlit\npython-docx\nreportlab\nPyMuPDF\n", "Dockerfile": "FROM python:3.11-slim\nWORKDIR /app\nCOPY requirements.txt /app/\nRUN apt-get update && apt-get install -y --no-install-recommends build-essential curl libgl1 && rm -rf /var/lib/apt/lists/*\nRUN pip install --no-cache-dir -r /app/requirements.txt\nCOPY . /app\nEXPOSE 8501\nCMD [\"streamlit\",\"run\",\"app.py\",\"--server.port=8501\",\"--server.address=0.0.0.0\"]\n", ".env": "LLM_BACKEND=dummy\nLLM_FINAL_BACKEND=same\nOUTPUT_DIR=/mnt/data/output\n", "README.md": "# mom_app\n\nDemo multi-agent MoM generator.\n"}
for p,c in files.items():
    full=os.path.join(base,p)
    os.makedirs(os.path.dirname(full),exist_ok=True)
    with open(full,'w',encoding='utf-8') as f:
        f.write(c)
print('Wrote',len(files),'files to',base)


## Section 3 — Create ZIP

In [ ]:

import zipfile, os
base='/mnt/data/mom_app'
zip_path='/mnt/data/mom_app.zip'
with zipfile.ZipFile(zip_path,'w',zipfile.ZIP_DEFLATED) as zf:
    for root,dirs,files in os.walk(base):
        for fname in files:
            full=os.path.join(root,fname)
            arc=os.path.relpath(full,base)
            zf.write(full,arc)
print('Created',zip_path)


## Section 4 — Quick demo run (optional)

In [ ]:

# Create a tiny demo file and run pipeline
demo='/mnt/data/mom_app/uploads/demo.txt'
with open(demo,'w') as f: f.write('Meeting demo content for pipeline test.')
from mom_app.pipeline import run_pipeline as rp
docx,pdf,out = rp(demo,'demo1')
print('Outputs:',docx,pdf,out)
